In [1]:
import os
from glob import glob
from tqdm.notebook import tqdm
import json


import matplotlib.pyplot as plt
import numpy as np
import librosa
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import top_k_accuracy_score, accuracy_score

import re
import hazm
import unicodedata
import Levenshtein

from einops import rearrange, repeat
import textgrid

import torch
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
from torchaudio import transforms as T

from resnet12 import resnet12, wide_resnet12

In [2]:
SEGMENT_LENGTH = 0.7
SHIFT_LENGTH = 0.05
N_MFCC = int(SEGMENT_LENGTH / 0.01)

N_SUPPORT = 10

SPEECH_TEMPLATE = "../Data/cv-corpus/fa/clips/{}.mp3"
textgrid_path = "../Data/alignments/fa/mnt/disks/std750/data/common-voice-forced-alignments/fa/alignments"

THRESHOLD = 0.9
MODEL_PATH = "./SAVED_MODELS/WideResNet12WithEMA.pth"

In [3]:
import json

with open('FILENAME_DICT.json', 'r') as fp:
    STREAM_EVAL_DICT = json.load(fp)

STREAM_EVAL_DICT = {"fa": STREAM_EVAL_DICT["fa"]}

In [4]:
normalizer = hazm.Normalizer(persian_style=False)

translation_src = "إ٫ٲٳٴڃڄﭪﭬﯔﯕﯖﯗﯘﯙﯞﯧﯨﯩﯼﯽﯾﯿﴽﺓةﺔۀأؤیؠػػؽؾؿكيٮٯٷٸٹٺٻټٽٿڀځٵٶٷٸٹٺٻټٽٿڀځڂڅڇڈډڊڋڌڍڎڏڐڑڒړڔڕږڗڙښڛڜڝڞڟڠڡڢڣڤڥڦڧڨڪګڬڭڮڰڱڲڳڴڵڶڷڸڹںڻڼڽھڿہۂۃۄۅۆۇۈۉۊۋۏۍێېۑےۓەۮۯۺۻۼۿݐݑݒݓݔݕݖݗݘݙݚݛݜݝݞݟݠݡݢݣݤݥݦݧݨݩݪݫݬݭݮݯݰݱݲݳݴݵݶݷݸݹݺݻݼݽݾݿࢠࢡࢢࢣࢤࢥࢦࢧࢨࢩࢪࢫࢮࢯࢰࢱࢬࢲࢳࢴࢶࢷࢸࢹࢺࢻࢼࢽﭐﭑﭒﭓﭔﭕﭖﭗﭘﭙﭚﭛﭜﭝﭞﭟﭠﭡﭢﭣﭤﭥﭦﭧﭨﭩﭮﭯﭰﭱﭲﭳﭴﭵﭶﭷﭸﭹﭺﭻﭼﭽﭾﭿﮀﮁﮂﮃﮄﮅﮆﮇﮈﮉﮊﮋﮌﮍﮎﮏﮐﮑﮒﮓﮔﮕﮖﮗﮘﮙﮚﮛﮜﮝﮞﮟﮠﮡﮢﮣﮤﮥﮦﮧﮨﮩﮪﮫﮬﮭﮮﮯﮰﮱﺀﺁﺃﺄﺅﺆﺇﺈﺉﺊﺋﺌﺍﺎﺏﺐﺑﺒﺕﺖﺗﺘﺙﺚﺛﺜﺝﺞﺟﺠﺡﺢﺣﺤﺥﺦﺧﺨﺩﺪﺫﺬﺭﺮﺯﺰﺱﺲﺳﺴﺵﺶﺷﺸﺹﺺﺻﺼﺽﺾﺿﻀﻁﻂﻃﻄﻅﻆﻇﻈﻉﻊﻋﻌﻍﻎﻏﻐﻑﻒﻓﻔﻕﻖﻗﻘﻙﻚﻛﻜﻝﻞﻟﻠﻡﻢﻣﻤﻥﻦﻧﻨﻩﻪﻫﻬﻭﻮﻯﻰﻱﻲﻳﻴىكي“” "
translation_dst = (
            'ا.ااءججفقکککوووویییییییاههههاوییککیییکیبقویتتبتتتبحاوویتتبتتتبحححچدددددددددررررررررسسسصصطعففففففققکککککگگگگگللللنننننهچهههوووووووووییییییهدرشضغهبببببببححددرسعععففکککممنننلررسححسرحاایییووییحسسکببجطفقلمییرودصگویزعکبپتریفقنااببببپپپپببببتتتتتتتتتتتتففففححححححححچچچچچچچچددددددددژژررککککگگگگگگگگگگگگننننننههههههههههییییءاااووااییییااببببتتتتثثثثججججححححخخخخددذذررززسسسسششششصصصصضضضضططططظظظظععععغغغغففففققققککککللللممممننننههههوویییییییکی"" '
        )

number_translation_src = "۰۱۲۳۴۵۶۷۸۹٪٠١٢٣٤٥٦٧٨٩"
number_translation_dst = "0123456789%0123456789"


normalizer.translation_src = translation_src
normalizer.translation_dst = translation_dst

normalizer.number_translation_src = number_translation_src
normalizer.number_translation_dst = number_translation_dst

In [5]:
with open("special_characters.txt", "r") as f:
    SPECIAL_CHARS = f.read().strip().split("\n")
    
def remove_special_characters(text, replace= ""):
    return re.sub('[' + re.escape("".join(SPECIAL_CHARS)) + ']', replace, text)

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [6]:
class SupportBatchSampler(Sampler):
    def __init__(self, filename_dict, n_support=10, min_folder=20):
        self.filename_dict = filename_dict
        self.n_support = n_support
        self.min_folder = min_folder

        self.filter_lang()
        self.LANG = list(self.filename_dict.keys())
        
        self.len_dataset = len(self)
        
    def __iter__(self):
        
        for lang in self.LANG:
            folders = self.filename_dict[lang]["folders"]
            for folder in folders:

                #batch_filenames = np.random.choice(self.filename_dict[lang]["filename_dict"][folder],
                #                                   size=self.n_support,
                #                                   replace=False).tolist()
                batch_filenames = self.filename_dict[lang]["filename_dict"][folder][:self.n_support]
                

                batch_labels = [folder] * self.n_support

                

                yield list(zip(batch_filenames, batch_labels))
            

    def __len__(self):
        return sum([len(self.filename_dict[lang]["folders"]) for lang in self.LANG])
    

    def filter_lang(self):
        delete_lang = []
        for lang in self.filename_dict.keys():
            if len(self.filename_dict[lang]["folders"]) < self.min_folder:
                delete_lang.append(lang)
                
        for lang in delete_lang:
            del self.filename_dict[lang]

In [7]:
support_sampler = SupportBatchSampler(STREAM_EVAL_DICT, n_support=N_SUPPORT)

In [8]:
for a in tqdm(support_sampler):
    pass

  0%|          | 0/851 [00:00<?, ?it/s]

In [9]:
class RemovePad(object):
    def __init__(self, thershold=0.05, fs=16_000, segment_move=0.025):
        self.thershold = thershold
        segment_move = int(fs * segment_move)
        self.window = np.ones(segment_move) / int(segment_move/2)

    def __call__(self, wav):
        moving_average_wav = np.convolve(np.abs(wav), self.window, mode='same')
        cutoff = np.where(moving_average_wav > self.thershold)[0]

        if len(cutoff) < 2:
            return wav

        return wav[cutoff[0]:cutoff[-1]]


class Normalize(object):
    def __init__(self):
        self.EPS = np.finfo(float).eps
        
    def __call__(self, wav):
        samples_99_percentile = np.percentile(np.abs(wav), 99.9)
        normalized_samples = wav / (samples_99_percentile + self.EPS)
        normalized_samples = np.clip(normalized_samples, -1, 1)
        return normalized_samples


class PadTrimWav(object):
    def __init__(self, segment_length=1.0, fs=16_000):
        self.segment_length = int(fs * segment_length)

    def __call__(self, wav):
        len_wav = len(wav)

        zeros = np.zeros(self.segment_length, dtype=np.float32)
        if len_wav < self.segment_length:
            start_point = (self.segment_length - len_wav) // 2
            zeros[start_point:start_point+len_wav] = wav
        else:
            start_point = (len_wav -  self.segment_length) // 2
            zeros = wav[start_point:start_point+self.segment_length]

        return zeros


class RandomPadTrimMFCC(object):
    def __init__(self, segment_shape=(40,100)):
        self.segment_shape = segment_shape

    def __call__(self, mfcc):
        mfcc = mfcc.numpy()
        mfcc_shape = mfcc.shape

        zeros = np.zeros(self.segment_shape, dtype=np.float32)
        if mfcc_shape[1] <= self.segment_shape[1]:
            random_point = np.random.randint(0, self.segment_shape[1] - mfcc_shape[1] + 1)
            zeros[:, random_point:random_point+mfcc_shape[1]] = mfcc
        else:
            random_point = np.random.randint(0, mfcc_shape[1] - self.segment_shape[1] + 1)
            zeros = mfcc[:, random_point:random_point+self.segment_shape[1]]

        return zeros


class PadTrimMFCC(object):
    def __init__(self, segment_shape=(40,100)):
        self.segment_shape = segment_shape

    def __call__(self, mfcc):
        mfcc = mfcc.numpy()
        mfcc_shape = mfcc.shape

        zeros = np.zeros(self.segment_shape, dtype=np.float32)
        if mfcc_shape[1] <= self.segment_shape[1]:
            start_point = (self.segment_shape[1] - mfcc_shape[1]) // 2
            zeros[:, start_point:start_point+mfcc_shape[1]] = mfcc
        else:
            start_point = (mfcc_shape[1] - self.segment_shape[1]) // 2
            zeros = mfcc[:, start_point:start_point+self.segment_shape[1]]

        return zeros


class ToTensor(object):
    def __call__(self, wav):
        return torch.from_numpy(wav)


class MFCC(object):
    def __init__(self, fs=16_000, n_fft=480, hop_length=160, n_mels=128, n_mfcc=41):
        
        self.mfcc = T.MFCC(
            sample_rate=fs,
            n_mfcc=n_mfcc,
            melkwargs={
                "n_fft": n_fft,
                "n_mels": n_mels,
                "hop_length": hop_length,
                "mel_scale": "htk",
            },
        )
        
    def __call__(self, wav):
        mfcc = self.mfcc(wav)[1:]
        mfcc = mfcc - mfcc.mean(axis=1, keepdims=True)
        return mfcc / 100.0


class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, wav):
        for transform in self.transforms:
            wav = transform(wav)

        return wav

In [10]:
class ReadWordDataset(Dataset):
    def __init__(self, fs=16_000, transforms=None):
        self.fs = fs
        self.transforms = transforms

    def __getitem__(self, idx):
        filename, label = idx
        wav, _ = librosa.load(filename, sr=self.fs)
        
        if self.transforms:
            wav = self.transforms(wav)

        return wav, label


class ReadSpeechDataset(Dataset):
    def __init__(self, keyword_filenames, 
                 textgrid_path,
                 speech_template,
                 sample_rate=16_000,
                 transforms=None,
                 split_length=1.0,
                 shift_length=0.05):

        self.find_textgrid(keyword_filenames, textgrid_path)
        self.speech_template = speech_template
        self.sample_rate = sample_rate
        self.transforms = transforms
        self.split_length = int(split_length * sample_rate)
        self.shift_length = int(shift_length * sample_rate)
        
        
    def __getitem__(self, idx):
        textgrid_filename = self.textgrid_filenames[idx]

        words = textgrid.TextGrid.fromFile(textgrid_filename)[0]
        for i in range(len(words)):
            word = normalizer.normalize(words[i].mark)
            word = strip_accents(word)
            word = remove_special_characters(word)
            words[i].mark = word
        
        
        basename = os.path.basename(textgrid_filename).split(".")[0]
        speech_filename = self.speech_template.format(basename)
        wav, _ = librosa.load(speech_filename, sr=self.sample_rate)

        segments = []
        if len(wav) > self.split_length:
            for idx in range(0, len(wav) - self.split_length, self.shift_length):
                segments.append(wav[idx : idx + self.split_length])
        else:
            segments.append(wav)

        segments_ = []
        for segment in segments:
            segment = self.transforms(segment)
            segments_.append(segment)

        return torch.stack(segments_, axis=0), words

    
    def __len__(self):
        return len(self.textgrid_filenames)


    def find_textgrid(self, keyword_filenames, textgrid_path):
        all_textgrid_filenames = glob(os.path.join(textgrid_path, "*/*.TextGrid"))
        all_textgrid_basenames = [os.path.basename(filename) for filename in all_textgrid_filenames]
        
        self.textgrid_filenames = []
        for keyword_filename in keyword_filenames:
            keyword_basename = os.path.basename(keyword_filename)
            if "__" in keyword_basename:
                keyword_basename = keyword_basename[:-8]
            else:
                keyword_basename = keyword_basename[:-5]
            keyword_basename = keyword_basename + ".TextGrid"
        
            if keyword_basename in all_textgrid_basenames:
                idx = all_textgrid_basenames.index(keyword_basename)
                self.textgrid_filenames.append(all_textgrid_filenames[idx])

In [11]:
support_transforms = Compose(
    [
        PadTrimWav(segment_length=SEGMENT_LENGTH),
        Normalize(),
        ToTensor(),
        MFCC(),
        PadTrimMFCC(segment_shape=(40,N_MFCC)),
        ToTensor(),
    ]
)

eval_support_dataset = ReadWordDataset(transforms=support_transforms)

/home/arya/miniconda3/envs/PyTorch/lib/python3.10/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (241) may be set too low.
  warnings.warn(


In [12]:
eval_transforms = Compose(
    [
        Normalize(),
        ToTensor(),
        MFCC(),
        PadTrimMFCC(segment_shape=(40,N_MFCC)),
        ToTensor(),
    ]
)

In [13]:
def support_collate_fn(x):
    batch_mfccs = []
    for mfcc, label in x:
        batch_mfccs.append(mfcc)

    batch_mfccs = torch.stack(batch_mfccs)
    return batch_mfccs, label

def search_collate_fn(x):
    return x

In [14]:
support_dataloader = DataLoader(eval_support_dataset,
                                batch_sampler=support_sampler,
                                num_workers=5,
                                collate_fn=support_collate_fn,
                                prefetch_factor=2)

In [15]:
class DummyModel(torch.nn.Module):
    def __init__(self, n_input=640, n_hidden=32):
        super().__init__()
        self.linear1 = torch.nn.Linear(n_input, n_hidden, bias=False)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(n_hidden, n_hidden, bias=False)
        self.linear3 = torch.nn.Linear(n_hidden, n_input, bias=False)
        

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x, _ = torch.max(x, axis=0, keepdim=True)
        x = self.linear3(x)
        return x



class WordSpotting(object):
    def __init__(self,
                 model_path,
                 gamma=3,
                 device="cuda"):
        
        self.model_path = model_path
        self.gamma = gamma
        self.device = device
        
        self.load_model()


    def __call__(self, x):
        with torch.no_grad():
            x = torch.unsqueeze(x, axis=1).to(self.device)
            x = self.model(x)
            
            return self.calculate_probability(x)


    def calculate_probability(self, output_query):
        distances = torch.cdist(output_query, self.centers, p=2)
        distances = -distances / self.center_weights
        likelihood = torch.exp(distances) / torch.exp(distances).sum(axis=1, keepdim=True)
        return likelihood


    def create_centers(self, support_samples):
        with torch.no_grad():
            support_samples = torch.unsqueeze(support_samples, axis=1).to(self.device)
            outputs = self.model(support_samples)

            support_center = rearrange(outputs, "(w s) e -> w s e", s=support_samples.size(0)).mean(axis=1)
            unknown_center = self.dummy_model(support_center)

            centers = torch.concat([support_center, unknown_center], axis=0)

        center_weights = torch.ones((1, centers.shape[0]))
        center_weights[:,-1] = self.gamma
        center_weights = center_weights.to(self.device)

        self.centers = centers
        self.center_weights = center_weights


    def load_model(self):
        model = wide_resnet12(normalize=False)
        model = model.to(self.device)

        dummy_model = DummyModel()
        dummy_model = dummy_model.to(self.device)

        checkpoint = torch.load(self.model_path)
        model.load_state_dict(checkpoint['model'])
        dummy_model.load_state_dict(checkpoint['dummy_model'])

        model.eval()
        dummy_model.eval()

        self.model = model
        self.dummy_model = dummy_model

In [16]:
WINDOW_LENGTH = int(SEGMENT_LENGTH / SHIFT_LENGTH)


def calculate_rate(predict, sentence, target_word):    
    idxs = []
    words_list = [word.mark for word in sentence]
    for word in words_list:
        if len(word) >= 3:
            dist = Levenshtein.distance(target_word, word, weights=(1,1,1))
            if (target_word in word) or (word in target_word):
                if dist <= 2:
                    idxs.append(words_list.index(word))

    
    hard_true_rate, soft_true_rate = [], []
    hard_false_rate, soft_false_rate = list(range(len(predict))), list(range(len(predict)))
    for idx in idxs:
        minTime, maxTime = sentence[idx].minTime, sentence[idx].maxTime
        minIdx, maxIdx = int(minTime / SHIFT_LENGTH), int(maxTime / SHIFT_LENGTH)
    
        if (maxIdx - minIdx) > WINDOW_LENGTH:
            maxIdx = minIdx + WINDOW_LENGTH
    
        hard_window_idx, soft_window_idx = [], []
        for i in range(len(predict)):
            hard_flag = (i <= minIdx) and ((i + WINDOW_LENGTH) >= maxIdx)
            soft_flag = (i <= (minIdx + (maxIdx - minIdx)//3)) and ((i + WINDOW_LENGTH) >= (maxIdx - (maxIdx - minIdx)//3))
            
            if hard_flag:
                hard_window_idx.append(i)
                if i in hard_false_rate:
                    hard_false_rate.remove(i)
    
            if soft_flag:
                soft_window_idx.append(i)
                if i in soft_false_rate:
                    soft_false_rate.remove(i)
                
        hard_true_rate.append([predict[i] for i in hard_window_idx])
        soft_true_rate.append([predict[i] for i in soft_window_idx])
    
    hard_false_rate = [predict[i] for i in hard_false_rate]
    soft_false_rate = [predict[i] for i in soft_false_rate]

    return hard_true_rate, soft_true_rate, hard_false_rate, soft_false_rate

In [ ]:
word_spotting = WordSpotting(model_path=MODEL_PATH)

counter = 0
FALSE_RATE, TRUE_RATE, BALANCE_TRUE_RATE = [], [], []
for S in tqdm(support_dataloader):
    word_spotting.create_centers(S[0])

    eval_search_dataset = ReadSpeechDataset(STREAM_EVAL_DICT["fa"]["filename_dict"][S[1]],
                                            textgrid_path,
                                            SPEECH_TEMPLATE,
                                            transforms=eval_transforms,
                                            split_length=SEGMENT_LENGTH,
                                            shift_length=SHIFT_LENGTH)
    
    search_dataloader = DataLoader(eval_search_dataset,
                                   batch_size=32,
                                   num_workers=8,
                                   shuffle=False,
                                   collate_fn=search_collate_fn,
                                   prefetch_factor=2)

    per_word_true_rate = []
    for Q in tqdm(search_dataloader):
        for mfcc, sentence in Q:
            predict = word_spotting(mfcc)
            predict = predict[:,1].cpu().numpy() < THRESHOLD

            hard_true_rate, soft_true_rate, hard_false_rate, soft_false_rate = calculate_rate(predict, sentence, S[1])

            TRUE_RATE += hard_true_rate
            FALSE_RATE += soft_false_rate
            per_word_true_rate += hard_true_rate
            
            

    true_rate = np.mean([part.count(True) > 0 for part in TRUE_RATE])
    true_rate = round(100 * np.mean(true_rate), 3)

    false_rate = 1 - np.mean(FALSE_RATE)
    false_rate = np.round(100 * false_rate, 3)

    per_word_true_rate = np.mean([part.count(True) > 0 for part in per_word_true_rate])
    BALANCE_TRUE_RATE.append(per_word_true_rate)
    per_word_true_rate = round(100 * np.mean(BALANCE_TRUE_RATE), 3)
    
    print(f"False Rate : {false_rate}, True Rate : {true_rate}, Balance True Rate : {per_word_true_rate}")

  0%|          | 0/851 [00:00<?, ?it/s]

/home/arya/miniconda3/envs/PyTorch/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


  0%|          | 0/6 [00:00<?, ?it/s]

False Rate : 99.839, True Rate : 77.957, Balance True Rate : 77.957


  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

False Rate : 99.165, True Rate : 64.334, Balance True Rate : 73.32


  0%|          | 0/11 [00:00<?, ?it/s]

False Rate : 99.165, True Rate : 64.373, Balance True Rate : 73.347


  0%|          | 0/89 [00:00<?, ?it/s]